In [596]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams["figure.facecolor"] = 'FFFFFF'
matplotlib.rcParams['figure.dpi'] = 100

In [597]:
data = pd.read_excel('online_retail_II.xlsx', [0, 1])

KeyboardInterrupt: 

In [ ]:
data = pd.concat([data[0], data[1]], ignore_index=True)

In [ ]:
print(data.columns)
pairs = 0
for i, row in enumerate(data[data['Invoice'].str[0] == 'C'].values):
    duplicates_found = data[data['StockCode'] == row[1]][data['Customer ID'] == row[6]][data['Quantity'] == -row[3]]
    if(len(duplicates_found) != 2):
        print(f'#{i}')
        print(row)
        print(duplicates_found)
        print(f"{pairs} / {i}")
        print('\n\n\n')
    else:
        pairs += 1

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Cancelled'],
      dtype='object')
#0
['C489449' 22087 'PAPER BUNTING WHITE LACE' -12
 Timestamp('2009-12-01 10:33:00') 2.95 16321.0 'Australia' True]
    Invoice StockCode               Description  Quantity         InvoiceDate  \
187  489450     22087  PAPER BUNTING WHITE LACE        12 2009-12-01 10:36:00   

     Price  Customer ID    Country  Cancelled  
187   2.95      16321.0  Australia      False  
0 / 0




#1
['C489449' '85206A' 'CREAM FELT EASTER EGG BASKET' -6
 Timestamp('2009-12-01 10:33:00') 1.65 16321.0 'Australia' True]
    Invoice StockCode                   Description  Quantity  \
188  489450    85206A  CREAM FELT EASTER EGG BASKET         6   

            InvoiceDate  Price  Customer ID    Country  Cancelled  
188 2009-12-01 10:36:00   1.65      16321.0  Australia      False  
0 / 1




#2
['C489449' 21895 "POTTING SHED SOW 'N' GROW SET" -4
 Timestamp

KeyboardInterrupt: 

In [ ]:
data['Cancelled'] = (data['Invoice'].str[0] == 'C')

In [ ]:
data.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Cancelled'],
      dtype='object')

In [ ]:
data['Cancelled'].value_counts()
data[['Invoice', 'Cancelled']].drop_duplicates()['Cancelled'].value_counts()

False    45336
True      8292
Name: Cancelled, dtype: int64

In [ ]:
data['StockCode'].value_counts()

85123A       5829
22423        4424
85099B       4216
21212        3318
20725        3259
             ... 
DCGS0067        1
DCGS0066P       1
DCGS0071        1
DCGS0073        1
90197A          1
Name: StockCode, Length: 5305, dtype: int64

In [ ]:
grouped_data = data.groupby('Invoice')

In [625]:
aggregated = {
    "Invoice": [],
    "EarliestInvoiceDate": [],
    "LatestInvoiceDate": [],
    "PriceSum": [],
    "TotalQuantity": [],
    "DifferentProducts": [],
    "Product1Quantity": [],
    "Product2Quantity": [],
    "Product3Quantity": [],
    "DifferentProducts": [],
    "Manual": [],
    "Discount": [],
    "ManualSum": [],
    "DiscountSum": [],
    "Cancelled": [],
}

for group_name, df_group in grouped_data:
    if all(df_group['StockCode'].isin(['M', 'm', 'S', 'B', 'D'])):
        continue
    assert not any(df_group['StockCode'] == 'S')

    assert all(df_group['Invoice'] == group_name)
    aggregated['Invoice'].append(group_name)

    group_invoice_dates = df_group['InvoiceDate'].sort_values()
    aggregated['EarliestInvoiceDate'].append(group_invoice_dates.iloc[0])
    aggregated['LatestInvoiceDate'].append(group_invoice_dates.iloc[-1])

    aggregated['PriceSum'].append((df_group['Price'] * df_group['Quantity']).abs().sum())
    aggregated['TotalQuantity'].append(df_group['Quantity'].abs().sum())
    aggregated['DifferentProducts'].append(df_group['StockCode'].isin(['M', 'm', 'D']).value_counts()[False])

    aggregated['Manual'].append(0)
    aggregated['Discount'].append(0)
    aggregated['ManualSum'].append(0)
    aggregated['DiscountSum'].append(0)

    stock_code_idx = list(df_group.columns).index('StockCode')
    price_idx = list(df_group.columns).index('Price')
    
    for row in df_group.values:
        if row[stock_code_idx] in {'M', 'm'}:
            aggregated['Manual'][-1] += 1
            aggregated['ManualSum'][-1] += row[price_idx]
        elif row[stock_code_idx] == 'D':
            aggregated['Discount'][-1] += 1
            aggregated['DiscountSum'][-1] += row[price_idx]
        
    aggregated['PriceSum'][-1] -= aggregated['DiscountSum'][-1] * 2

    aggregated['Cancelled'].append(str(group_name)[0] == 'C')

In [626]:
old_agg_data = pd.DataFrame(aggregated)
old_agg_data.columns

Index(['Invoice', 'EarliestInvoiceDate', 'LatestInvoiceDate', 'PriceSum',
       'TotalQuantity', 'DifferentProducts', 'Manual', 'Discount', 'ManualSum',
       'DiscountSum', 'Cancelled'],
      dtype='object')

In [627]:
print(len(old_agg_data))
print(len(old_agg_data.drop_duplicates(subset=['PriceSum', 'TotalQuantity'], keep=False)))
# old_agg_data = old_agg_data.drop_duplicates(subset=['PriceSum'], keep=False)
# old_agg_data = old_agg_data.drop_duplicates(subset=['TotalQuantity'], keep=False)
print(len(old_agg_data))
agg_data = old_agg_data

# agg_data = old_agg_data.drop_duplicates(subset=['PriceSum', 'TotalQuantity'], keep=False)

52734
38203
52734


In [628]:
ready_data = agg_data.drop(columns=['Invoice', 'EarliestInvoiceDate', 'LatestInvoiceDate'])
# 'Manual', 'ManualSum', 'Discount', 'DiscountSum', 'EarliestInvoiceDate', 'LatestInvoiceDate', 'PriceSum'
ready_data['InvoiceDate'] = agg_data['EarliestInvoiceDate'].dt.month * 31 + agg_data['EarliestInvoiceDate'].dt.day
ready_data['InvoiceHour'] = agg_data['EarliestInvoiceDate'].dt.hour

print(agg_data['EarliestInvoiceDate'])

0       2009-12-01 07:45:00
1       2009-12-01 07:46:00
2       2009-12-01 09:06:00
3       2009-12-01 09:08:00
4       2009-12-01 09:24:00
                ...        
52729   2011-12-08 19:28:00
52730   2011-12-09 09:27:00
52731   2011-12-09 09:57:00
52732   2011-12-09 11:57:00
52733   2011-12-09 11:58:00
Name: EarliestInvoiceDate, Length: 52734, dtype: datetime64[ns]


In [629]:
train_data, valid_data = train_test_split(ready_data, random_state=421)
train_X, train_y = train_data.drop(columns=['Cancelled']), train_data['Cancelled']
valid_X, valid_y = valid_data.drop(columns=['Cancelled']), valid_data['Cancelled']

In [630]:
# plt.hist(train_data['TimeSpent'], bins=list(range(0, 20, 1)));

In [631]:
train_data.corr()

,PriceSum,TotalQuantity,DifferentProducts,Manual,Discount,ManualSum,DiscountSum,Cancelled,InvoiceDate,InvoiceHour
PriceSum,1.000000,0.683048,0.279383,0.005087,-0.000041,0.013184,0.002577,-0.068241,0.030764,-0.022327
TotalQuantity,0.683048,1.000000,0.127947,0.001498,0.000260,-0.001458,0.002452,-0.072779,0.010259,-0.024220
DifferentProducts,0.279383,0.127947,1.000000,0.059230,-0.003453,0.008162,-0.002823,-0.174802,0.048974,0.042190
Manual,0.005087,0.001498,0.059230,1.000000,-0.001021,0.185063,-0.000960,-0.000392,0.002803,0.021623
Discount,-0.000041,0.000260,-0.003453,-0.001021,1.000000,-0.000239,0.850606,0.025596,0.000711,0.008754
ManualSum,0.013184,-0.001458,0.008162,0.185063,-0.000239,1.000000,-0.000225,0.007796,0.002077,0.002480
DiscountSum,0.002577,0.002452,-0.002823,-0.000960,0.850606,-0.000225,1.000000,0.024076,0.004615,0.006511
Cancelled,-0.068241,-0.072779,-0.174802,-0.000392,0.025596,0.007796,0.024076,1.000000,-0.009605,0.060512
InvoiceDate,0.030764,0.010259,0.048974,0.002803,0.000711,0.002077,0.004615,-0.009605,1.000000,0.017435
InvoiceHour,-0.022327,-0.024220,0.042190,0.021623,0.008754,0.002480,0.006511,0.060512,0.017435,1.000000


In [632]:
train_data['InvoiceDate'].value_counts()

373    348
374    339
380    334
345    283
379    276
      ... 
127     15
156     15
75      13
106     12
68       9
Name: InvoiceDate, Length: 352, dtype: int64

In [633]:
model = xgb.XGBClassifier()
model.fit(train_X, train_y)
pred = model.predict(valid_X)
# print(train_y)

print(cohen_kappa_score(valid_y, pred))
print(accuracy_score(valid_y, pred))
# print(export_text(model))

0.6910643358520587
0.9241504854368932


In [634]:
print(len(valid_y))

13184


In [635]:
rand = (np.random.uniform(0, 1, len(valid_y)) > train_data['Cancelled'].value_counts()[False] / len(train_data['Cancelled'])).astype(np.int64)
print(cohen_kappa_score(valid_y, rand))
print(accuracy_score(valid_y, rand))

-0.008691901122262369
0.7505309466019418


In [636]:
zeros = np.zeros(len(valid_y))
print(cohen_kappa_score(valid_y, zeros))
print(accuracy_score(valid_y, zeros))

0.0
0.8596783980582524
